In [1]:
!pip install numpy pandas matplotlib seaborn scikit-learn tensorflow transformers tf-keras
!pip install pyarrow
import os
import tensorflow as tf
tf.compat.v1.losses.sparse_softmax_cross_entropy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
from sklearn.preprocessing import LabelEncoder
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [2]:
import pandas as pd

# File paths
file_paths = {
    "amazon_polarity_train": "C:/Project/dataset_cache/amazon_polarity_train.parquet",
    "amazon_polarity_test": "C:/Project/dataset_cache/amazon_polarity_test.parquet",
    "go_emotions_train": "C:/Project/dataset_cache/go_emotions_train.parquet",
    "go_emotions_test": "C:/Project/dataset_cache/go_emotions_test.parquet",
    "go_emotions_validation": "C:/Project/dataset_cache/go_emotions_validation.parquet",
    "imdb_train": "C:/Project/dataset_cache/imdb_train.parquet",
    "imdb_test": "C:/Project/dataset_cache/imdb_test.parquet",
    "imdb_unsupervised": "C:/Project/dataset_cache/imdb_unsupervised.parquet"
}

# Loading datasets into Pandas DataFrames
dataframes = {name: pd.read_parquet(path) for name, path in file_paths.items()}
print("Datasets loaded successfully!")


Datasets loaded successfully!


In [5]:
import nltk
import os

# Setting project folder as the NLTK data path
project_folder = "C:/Project/nltk_data"
os.makedirs(project_folder, exist_ok=True)
nltk.data.path.append(project_folder)

# Download required datasets directly to the project folder
nltk.download('punkt', download_dir=project_folder)
nltk.download('stopwords', download_dir=project_folder)
nltk.download('wordnet', download_dir=project_folder)
nltk.download('omw-1.4', download_dir=project_folder)

print(" NLTK resources installed to:", project_folder)



[nltk_data] Downloading package punkt to C:/Project/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:/Project/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:/Project/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:/Project/nltk_data...


 NLTK resources installed to: C:/Project/nltk_data


[nltk_data]   Package omw-1.4 is already up-to-date!


In [27]:
import nltk

# Clear existing paths and setting the project path
nltk.data.path.clear()
nltk_path = "C:/Project/nltk_data"
nltk.data.path.append(nltk_path)

print("Updated NLTK Path:", nltk.data.path)


Updated NLTK Path: ['C:/Project/nltk_data']


In [49]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    try:
        # Converting to lowercase
        text = text.lower()

        # Removing special characters and numbers
        text = re.sub(r'[^a-z\s]', '', text)

        # Tokenization using regex
        tokens = re.findall(r'\b\w+\b', text)

        # Removing stopwords and applying lemmatization
        processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

        # Joins tokens back to a cleaned sentence
        cleaned_text = " ".join(processed_tokens)
        return cleaned_text
    
    except Exception as e:
        print(f"Error processing text: {e}")
        return text

def preprocess_loaded_data(dataframes):
    try:
        for name, df in dataframes.items():
            print(f"Processing {name} data...")

            if 'content' in df.columns:
                df['cleaned_text'] = df['content'].apply(preprocess_text)
                print(f"{name.capitalize()} data processed successfully!\n")
            else:
                print(f"Skipping {name} as it has no 'content' column.\n")
        
        print("Preprocessing completed for all datasets.")
    except Exception as e:
        print(f"Error during preprocessing: {e}")


[nltk_data] Downloading package punkt to C:\Project\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Project\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Project\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
preprocess_loaded_data(dataframes)



In [33]:
sample_text = "Hello darling! I love talking to you every single day. Isn't it wonderful?"
print("Original Text:", sample_text)

cleaned_text = preprocess_text(sample_text)
print("Cleaned Text:", cleaned_text)


Original Text: Hello darling! I love talking to you every single day. Isn't it wonderful?
Original Text: Hello darling! I love talking to you every single day. Isn't it wonderful?...
Lowercased Text: hello darling! i love talking to you every single day. isn't it wonderful?...
Cleaned Text (No Special Chars & Numbers): hello darling i love talking to you every single day isnt it wonderful...
Tokens: ['hello', 'darling', 'i', 'love', 'talking', 'to', 'you', 'every', 'single', 'day', 'isnt', 'it', 'wonderful']...
Processed Tokens (After Stopword Removal & Lemmatization): ['hello', 'darling', 'love', 'talking', 'every', 'single', 'day', 'isnt', 'wonderful']...
Final Cleaned Text: hello darling love talking every single day isnt wonderful...

Cleaned Text: hello darling love talking every single day isnt wonderful


In [53]:
for name, df in dataframes.items():
    df.to_parquet(f"C:/Project/dataset_cache/{name}_preprocessed.parquet")
    print(f"{name} saved successfully as Parquet!")


amazon_polarity_train saved successfully as Parquet!
amazon_polarity_test saved successfully as Parquet!
go_emotions_train saved successfully as Parquet!
go_emotions_test saved successfully as Parquet!
go_emotions_validation saved successfully as Parquet!
imdb_train saved successfully as Parquet!
imdb_test saved successfully as Parquet!
imdb_unsupervised saved successfully as Parquet!
